# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
#     df = pd.read_csv(f)
#     for i, row in df.iterrows():
#         csv_row = (row.artist, row.auth, row.firstName, row.gender, row.itemInSession, \
#                    row.lastName, row.length, row.level, row.location, row.method, row.page, \
#                    row.registration, row.sessionId, row.song, row.status, row.ts, row.userId)
#         full_data_rows_list.append(csv_row)

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader) 
        # extracting each data row one by one and append it        
        for line in csvreader:
#             print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)



with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


    


2533


In [4]:
# checks the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

2125


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Creates an idempotent Keyspace 
try:
    session.execute("""DROP KEYSPACE IF EXISTS user_plays""")
except Exception as e:
    print(e)
    
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS user_plays
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor' : 1 }""")
    
except Exception as e:
        print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to user_plays specified above
try:
    session.set_keyspace('user_plays')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4
query = "drop table filter_by_session_id_and_item_in_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Partiion key selected as session_id and item_in_session to make key unique, \
# no clustering column needed as no sorting has been requested
query = "CREATE TABLE IF NOT EXISTS filter_by_session_id_and_item_in_session "
query = query + "(session_id int, item_in_session int, artist text, song text, length float, PRIMARY KEY ((session_id, item_in_session)))"

try:
    session.execute(query)
except Exception as e:
    print(e)      

Error from server: code=2200 [Invalid query] message="unconfigured table filter_by_session_id_and_item_in_session"


In [20]:
# Reads our denormalized CSV file and inserts it into Cassandra table
file = 'event_datafile_new.csv'

# Add dataframe columns to query output
df = pd.read_csv(file)
for i, row in df.iterrows():
    query = "INSERT INTO filter_by_session_id_and_item_in_session(session_id, item_in_session, artist, song, length) "
    query = query + "VALUES(%s, %s, %s, %s, %s)"
    # Assign columns from csv to appropriate columns of table
    session.execute(query, (row.sessionId, row.itemInSession, row.artist, row.song, row.length))

# with open(file, encoding = 'utf8') as f:
#     csvreader = csv.reader(f)
#     next(csvreader) # skip header
#     for line in csvreader:
#         query = "INSERT INTO filter_by_session_id_and_item_in_session(session_id, item_in_session, artist, song, length) "
#         query = query + "VALUES(%s, %s, %s, %s, %s)"
#         # Assign columns from csv to appropriate columns of table and converts data type from received string
#         session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [21]:
# Use where clause on session_id and add item_in_session to find unique row
query = "select * from filter_by_session_id_and_item_in_session WHERE SESSION_ID=338 AND ITEM_IN_SESSION=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
        print (row.artist, row.song, row.length)


Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [22]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "drop table if exists filter_by_user_id_and_session_id"

try:
    session.execute(query)
except Exception as e:
    print(e)

# Partition key is user_id and session_id in event same song is played in a different session by same user, \
# cluster columns were determined since songs must be sorted by item_in_session
query = "CREATE TABLE IF NOT EXISTS filter_by_user_id_and_session_id "
query = query + "(user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text, PRIMARY KEY ((user_id, session_id), song, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)    

In [23]:
# Reads our denormalized CSV file and inserts it into Cassandra table
file = 'event_datafile_new.csv'

# Add dataframe columns to query output 
df = pd.read_csv(file)
for i, row in df.iterrows():
    query = "INSERT INTO filter_by_user_id_and_session_id(user_id, session_id, item_in_session, artist, song, first_name, last_name) "
    query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
    # Assign columns from csv to appropriate columns of table
    session.execute(query, (row.userId, row.sessionId, row.itemInSession, row.artist, row.song, row.firstName, row.lastName))

# Built-in csv read into query
# with open(file, encoding = 'utf8') as f:
#     csvreader = csv.reader(f)
#     next(csvreader) # skip header
#     for line in csvreader:
#     Assign the INSERT statements into the `query` variable
#         query = "INSERT INTO filter_by_user_id_and_session_id(user_id, session_id, item_in_session, artist, song, first_name, last_name) "
#         query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
#         # Assign columns from csv to appropriate columns of table and converts data type from received string
#         session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [24]:
# Reads each line of denormalized CSV file and inserts it into Cassandra table

query = "select * from filter_by_user_id_and_session_id WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:         
    print (row.artist, row.song, row.first_name, row.last_name)

Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Down To The Bone Keep On Keepin' On Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz


In [25]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# Drop makes this table idempotent
query = "DROP TABLE IF EXISTS filter_by_song"
try:
    session.execute(query)
except Exception as e:
    print(e)

# Partitioning by song alone is fine, we can sort by user_id to make the primary key unique
query = "CREATE TABLE IF NOT EXISTS filter_by_song "
query = query + "(song text, user_id int, first_name text, last_name text, PRIMARY KEY (song, user_id))"


try:
    session.execute(query)
except Exception as e:
    print(e)        

In [27]:
# Reads each line of denormalized CSV file and inserts it into Cassandra table
file = 'event_datafile_new.csv'

df = pd.read_csv(file)
for i, row in df.iterrows():
    query = "INSERT INTO filter_by_song(song, user_id, first_name, last_name) "
    query = query + "VALUES(%s, %s, %s, %s)"
    # Assign columns from csv to appropriate columns of table
    session.execute(query, (row.song, row.userId, row.firstName, row.lastName))


# with open(file, encoding = 'utf8') as f:
#     csvreader = csv.reader(f)
#     next(csvreader)
#     for line in csvreader:
#         query = "INSERT INTO filter_by_song(song, user_id, first_name, last_name) "
#         query = query + "VALUES(%s, %s, %s, %s)"
#         # Assign columns from csv to appropriate columns of table and converts data type from received string
#         session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [28]:
# A secondary index isn't needed since we partitioned on song and user_id
query = "select * from filter_by_song WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# We only have one clustering column so we don't need to worry about putting results it in order
for row in rows:         
    print (row.first_name, row.last_name)

Jacqueline Lynch
Sara Johnson


### Drop the tables before closing out the sessions

In [29]:
# Drop all tables
query = "drop table filter_by_session_id_and_item_in_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table filter_by_user_id_and_session_id"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table filter_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [30]:
# Close the connection to database
session.shutdown()
cluster.shutdown()